<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_Parte3_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## C++

# Código

In [1]:
%%writefile comm_data_types.h
#ifndef _COMM_DATA_TYPES_H_
#define _COMM_DATA_TYPES_H_

#define  CLIENT_FIFO_PATH "./client_fifo"
#define  SERVER_FIFO_PATH "./server_fifo"

typedef struct
{
    int total_sum;
    int num_quantity;
    float average;
    int min;
    int max;
} ServerResponse;

typedef struct
{
    int size;
    int numbers[100];
} ClientRequest;

#endif

Writing comm_data_types.h


In [2]:
%%writefile client.cpp
#include <iostream>
#include <string>
#include <vector>
#include <sstream>
#include <sys/types.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <fcntl.h>
#include <unistd.h>
#include "comm_data_types.h"

std::vector<int> ParseInput(const std::string& argv);
void PrintResponse(ServerResponse& response);

int main(int argc, char* argv[])
{
    if(argc < 2)
    {
        std::cout << "Cantidad de atributos erronea" << std::endl;
        return EXIT_FAILURE;
    }

    std::string input(argv[1]);
    auto numbers {ParseInput(input)};

    // write
    ClientRequest request {(int)numbers.size()};
    std::copy(numbers.begin(), numbers.end(), request.numbers);
    
    int w = open(CLIENT_FIFO_PATH, O_WRONLY);
    if( write( w, &request, sizeof( ClientRequest ) ) == -1 )
    {
        perror("Error en write()");
        return EXIT_FAILURE;
    }    
    
    close(w);
    unlink(CLIENT_FIFO_PATH);

    ServerResponse response;

    // Read
    int r = open(SERVER_FIFO_PATH, O_RDONLY);
    if( read( r, &response, sizeof( ServerResponse ) ) == -1 )
    {
        perror("Error en read()");
        return EXIT_FAILURE;
    }
    close(r);

    PrintResponse(response);

    return EXIT_SUCCESS;
}

std::vector<int> ParseInput(const std::string& input)
{
    std::vector<int> numbers;
    
    std::stringstream stream(input);
    std::string segment;
    std::vector<std::string> seglist;

    while(std::getline(stream, segment, '-'))
    {
        seglist.push_back(segment);
    }

    for(const std::string s: seglist)
    {
        numbers.push_back(std::stoi(s));
    }

    return numbers;
}

void PrintResponse(ServerResponse& response)
{
    std::cout 
        << "Promedio: " << response.average << std::endl
        << "Min: " << response.min << std::endl
        << "max: " << response.max << std::endl
        << "Suma total: " << response.total_sum << std::endl
        << "Cantidad números: " << response.num_quantity << std::endl;
}

Writing client.cpp


In [3]:
%%writefile server.cpp
#include <iostream>
#include <vector>
#include <numeric>
#include <algorithm>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include "comm_data_types.h"

ServerResponse CalculateResponse(std::vector<int>&);

int main(int argc, char* argv[])
{
    // Read
    if( mkfifo(CLIENT_FIFO_PATH,0666) == -1 )
    {
        perror("Error al crear client fifo");
        return EXIT_FAILURE;    
    }
    int r = open(CLIENT_FIFO_PATH, O_RDONLY);
    ClientRequest request {};
    
    if( read( r, &request, sizeof( ClientRequest ) ) == -1 )
    {
        perror("Error en read()");
        return EXIT_FAILURE;
    }

    close(r);

    std::vector<int> numbers;
    for(int i = 0; i < request.size; ++i)
    {
        numbers.push_back(request.numbers[i]);
    }

    ServerResponse response {CalculateResponse(numbers)};

    if( mkfifo(SERVER_FIFO_PATH,0666) == -1 )
    {
        perror("Error al crear ./server_fifo");
        return EXIT_FAILURE;    
    }

    int w = open(SERVER_FIFO_PATH, O_WRONLY);

    if( write( w, &response, sizeof( ServerResponse ) ) == -1 )
    {
        perror("Error en write()");
        return EXIT_FAILURE;
    }

    close(w);
    unlink(SERVER_FIFO_PATH);

    return EXIT_SUCCESS;
}

ServerResponse CalculateResponse(std::vector<int> &numbers)
{
    // std::reduce Calcula la sumatoria de elementos de un vector de una manera concurrente
    int total_sum {std::reduce(numbers.begin(), numbers.end())};
    int num_quantity {(int)numbers.size()};
    ServerResponse respuesta 
    {
        total_sum, // total_sum
        num_quantity, // num_quantity
        (float)total_sum/num_quantity, // average
        // Devuelven vector<int>::iterator, que es un puntero a un elemento de un vector
        // Por ende, hay que desferrenciar
        *std::min_element(numbers.begin(), numbers.end()), // min
        *std::max_element(numbers.begin(), numbers.end()) // max
    };

    return respuesta;
}

Writing server.cpp


In [4]:
%%writefile makefile
all: server client

server:./server.cpp
	g++ -std=c++17 ./server.cpp -o server

client:./client.cpp
	g++ -std=c++17 ./client.cpp -o client

clean:
	rm server
	rm client

Writing makefile


# Ejecución

In [5]:
!make

g++ -std=c++17 ./server.cpp -o server
g++ -std=c++17 ./client.cpp -o client


In [6]:
!nohup ./server &

nohup: appending output to 'nohup.out'


In [7]:
!./client 222-444-55-6

Promedio: 181.75
Min: 6
max: 444
Suma total: 727
Cantidad números: 4


In [ ]:
!rm -f server_fifo client_fifo